<a href="https://colab.research.google.com/github/MelikbekyanAshot/Data-Science/blob/main/MartirosSarianGAN/MartirosSarianGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install loguru

     |████████████████████████████████| 58 kB 3.8 MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup

import random
from loguru import logger

import pandas as pd
import numpy as np

from collections import Counter

In [ ]:
Counter([1, 1, 1, 2, 2])[1]

3

In [ ]:
class Settings:
  @staticmethod
  def set_logging_settings(path: str, name: str):
    """Настройка логгирования.
    
    Args:
      path (str): путь файла логгирования.
      name (str): название файла логгирования.
    """
    logger.remove()
    logger.add(path + '/' + name,
               format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {message}',
               backtrace=True, diagnose=True, enqueue=True)

In [ ]:
class PaintingDownloader:
  def __init__(self, paintings_urls=list(), paintings_names=list()):
    self.paintings_urls = paintings_urls
    self.paintings_names = paintings_names

  def get_pictures_urls(self):
    main_url = 'https://www.wikiart.org/en/martiros-sarian/all-works/text-list'
    main_page = requests.get(main_url)
    main_soup = BeautifulSoup(main_page.text)
    URLS = main_soup.find_all('li', {'class': 'painting-list-text-row'})

    for link in URLS:
      try:
        painting_page = requests.get('https://www.wikiart.org' + link.find('a').get('href'))
        painting_soup = BeautifulSoup(painting_page.text)

        if painting_soup.find('img', {'itemprop': 'image'}):
          name_ = painting_soup.find('img', {'itemprop': 'image'}).get('title')
          self.paintings_names.append(name_)
          url_ = painting_soup.find('img', {'itemprop': 'image'}).get('src')
          self.paintings_urls.append(url_)
          logger.success(f'Картина {name_} по ссылке {url_} загружена')
        else:
          logger.error(f'Не найдена картина по ссылке {link}')
      except Exception as e:
        logger.error(e)
  
  def download_pictures(self, path: str):
    for link, name in zip(self.paintings_urls, self.paintings_names):
      img_data = requests.get(link).content
      if Counter(self.paintings_names)[name] == 1:
        with open(f'{path}/{name}.jpg', 'wb') as handler:
          handler.write(img_data)
      else:
        with open(f'{path}/{name}({{random.randint(1, 10000)}}).jpg', 'wb') as handler:
          handler.write(img_data)

In [ ]:
Settings.set_logging_settings('/content/drive/MyDrive/Data Science/SarianGAN', 'log_file')

In [ ]:
paintingDownloader = PaintingDownloader()

In [ ]:
paintingDownloader.get_pictures_urls()

In [ ]:
paintingDownloader.paintings_names

In [ ]:
len(paintingDownloader.paintings_names)

551

In [ ]:
paintingDownloader.download_pictures(path='/content/drive/MyDrive/Data Science/SarianGAN/Data')